# **Preface**
In this notebook you are going to implement a powerful blackbox attack namely [NES](https://arxiv.org/pdf/1804.08598.pdf) and test it on a regular target model, then you will use a simple but powerful defense called  Random Noise Defense ([RND](https://arxiv.org/pdf/2104.11470.pdf)) against this attack on the same target model in order to check the power of the aforementioned attack in the presence of this so-called defense.
You will have to use a **CIFAR10 ResNet18 model** as the target model for this set of experiments.
You may want to train a new model or load an already trained model, your choice, but this model must have at least 94% accuracy on CIFAR10 test set as told in the previous exercises.

P.S. Don't normalize the data used for training.


# **Natural Evolutionary Strategies (NES)**

Firstly complete the code for the attack itself and then the add_Gaussian method used for the Random Noise Defense in the NES attack class.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision import models
import torchvision
import torchvision.transforms as transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import os

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [ ]:
'''Train CIFAR10 with PyTorch.'''





lr = 0.1
resume = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')

net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()

Streaming output truncated to the last 5000 lines.
32 100 Loss: 0.171 | Acc: 95.545% (3153/3300)
33 100 Loss: 0.171 | Acc: 95.500% (3247/3400)
34 100 Loss: 0.173 | Acc: 95.514% (3343/3500)
35 100 Loss: 0.175 | Acc: 95.500% (3438/3600)
36 100 Loss: 0.179 | Acc: 95.459% (3532/3700)
37 100 Loss: 0.180 | Acc: 95.368% (3624/3800)
38 100 Loss: 0.177 | Acc: 95.462% (3723/3900)
39 100 Loss: 0.179 | Acc: 95.425% (3817/4000)
40 100 Loss: 0.178 | Acc: 95.463% (3914/4100)
41 100 Loss: 0.178 | Acc: 95.429% (4008/4200)
42 100 Loss: 0.177 | Acc: 95.512% (4107/4300)
43 100 Loss: 0.177 | Acc: 95.523% (4203/4400)
44 100 Loss: 0.177 | Acc: 95.533% (4299/4500)
45 100 Loss: 0.179 | Acc: 95.522% (4394/4600)
46 100 Loss: 0.177 | Acc: 95.532% (4490/4700)
47 100 Loss: 0.182 | Acc: 95.438% (4581/4800)
48 100 Loss: 0.180 | Acc: 95.449% (4677/4900)
49 100 Loss: 0.184 | Acc: 95.400% (4770/5000)
50 100 Loss: 0.181 | Acc: 95.451% (4868/5100)
51 100 Loss: 0.181 | Acc: 95.462% (4964/5200)
52 100 Loss: 0.181 | Acc: 95.

In [6]:
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

checkpoint = torch.load('./checkpoint/ckpt.pth')
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [7]:
import torch
import numpy as np
import torch as ch
import torchvision
import torchvision.transforms.functional as F
import torchvision.transforms as transforms
import argparse
import json
import pdb
import os
import time
import math
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from torch.nn.modules import Upsample
from matplotlib import pyplot as plt

CLASSIFIERS = {
    "resnet18": (models.resnet50, 224),
}

NUM_CLASSES = {
    "cifar10": 10,
}

# Set the path to your dataset
CIFAR10_PATH = "/content/drive/MyDrive/myCIFAR10"

if CIFAR10_PATH == "":
  raise ValueError("Please fill out the path to Cifar10!")

class NES():
  
  def __init__(self, model_to_fool, args, dataset_size, RND_coefficient, device):


      if args != None:
        self.args = args
      else:
        self.max_queries = 10000
        self.fd_eta = 0.01
        self.image_lr = 0.01
        self.mode = "linf"
        self.json_config = None
        self.epsilon = 0.05
        self.batch_size = 10
        self.log_process = "store_true"
        self.gradient_iters = 15
        self.total_images = 10000
        self.classifier = 'resnet18'
        self.args = {'max_queries': self.max_queries,
          'fd_eta': self.fd_eta,
          'image_lr': self.image_lr,
          'mode': self.mode,
          'json_config': self.json_config,
          'epsilon': self.epsilon,
          'batch_size': self.batch_size,
          'log_progress': self.log_progress,
          'gradient_iters': self.gradient_iters,
          'total_images': self.total_images,
          'classifier': self.classifier,
          }
      self.model_to_fool = model_to_fool
      self.dataset_size = dataset_size
      self.RND_coefficient = RND_coefficient
      self.device = device
      if self.device == 'cuda':
        ch.set_default_tensor_type('torch.cuda.FloatTensor')
      else:
        ch.set_default_tensor_type('torch.FloatTensor')
      
      if self.dataset_size == 32:
        self.channel_size = 3
      else:
        raise ValueError("Please check out the dataset_size parameter...")
      
  
  """For setting up the Random Noise Defense"""

  def add_Gausian(self, input_image, coeff):
    # Firstly define a random epsilon with the input image shape
    # Remember that it should be a float
    # Add the epsilon with the proper coefficient to the input image
    # Don't forget to clamp it between the right values
    # P.S. Remember to convert everything to the right device
    #############################
    # Your code goes here
    epsilon = torch.randn_like(input_image).to(input_image.device)
    output_image = input_image + coeff * epsilon
    output_image = torch.clamp(output_image, 0, 1)
    return output_image

    #############################
    

  def generate(self):
      dataset = None
      if self.dataset_size == 32:
        transform = transforms.Compose([transforms.ToTensor(), transforms.Resize(self.dataset_size), transforms.CenterCrop(self.dataset_size),])
        dataset = torchvision.datasets.CIFAR10(root = CIFAR10_PATH, train = False, download = False, transform = transform)
      else:
        raise ValueError("Please check out the dataset_size parameter...")
      
      dataset_loader = DataLoader(dataset, batch_size=self.args["batch_size"])
      total_correct, total_adv, total_queries = 0, 0, 0
      for i, (images, targets) in enumerate(dataset_loader):
          if i*self.args["batch_size"] >= self.args["total_images"]:
              break

          res = self.make_adversarial_examples(images.to(self.device), targets.to(self.device), self.args, self.model_to_fool, self.dataset_size)
          ncc = res['num_correctly_classified'] # Number of correctly classified images (originally)
          num_adv = ncc * res['success_rate'] # Success rate was calculated as (# adv)/(# correct classified)
          queries = num_adv * res['average_queries'] # Average queries was calculated as (total queries for advs)/(# advs)
          total_correct += ncc
          total_adv += num_adv
          total_queries += queries
          print("i: " + str(i))
      print("-"*80)
      if total_adv != 0:
        print("Final Success Rate: {succ} | Final Average Queries for Successful Adv Examples: {aq}".format(
                aq=int(total_queries/total_adv),
                succ=total_adv/total_correct))
      else:
        """If total adversarial examples is zero then we won't print any avg number of queries for success
        because we had no success and the avg number of queries has exceeded the budget"""
        print("Final Success Rate: {succ} ".format(
                # aq = total_queries,
                succ=total_adv/total_correct))
      print("-"*80)
      #return self.adversarial_images

  def norm(self, t):
      assert len(t.shape) == 4
      norm_vec = ch.sqrt(t.pow(2).sum(dim=[1,2,3])).view(-1, 1, 1, 1)
      norm_vec += (norm_vec == 0).float()*1e-8
      return norm_vec

  ###
  # Different optimization steps
  # All take the form of func(x, g, lr)
  # eg: exponentiated gradients
  # gd: gradient descent
  # l2/linf: projected gradient descent
  ###


  def eg_step(self, x, g, lr):
      real_x = (x + 1)/2 # from [-1, 1] to [0, 1]
      pos = real_x*ch.exp(lr*g)
      neg = (1-real_x)*ch.exp(-lr*g)
      new_x = pos/(pos+neg)
      return new_x*2-1

  def linf_step(self, x, g, lr):
      return x + lr*ch.sign(g)

  def l2_prior_step(self, x, g, lr):
      new_x = x + lr*g/self.norm(g)
      norm_new_x = self.norm(new_x)
      norm_mask = (norm_new_x < 1.0).float()
      return new_x*norm_mask + (1-norm_mask)*new_x/norm_new_x

  def gd_prior_step(self, x, g, lr):
      return x + lr*g
    
  def l2_image_step(self, x, g, lr):
      return x + lr*g/self.norm(g)

  ##
  # Projection steps for l2 and linf constraints:
  # All take the form of func(new_x, old_x, epsilon)
  ##

  def l2_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          delta = new_x - orig
          out_of_bounds_mask = (self.norm(delta) > eps).float()
          x = (orig + eps*delta/self.norm(delta))*out_of_bounds_mask
          x += new_x*(1-out_of_bounds_mask)
          return x
      return proj

  def linf_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          return orig + ch.clamp(new_x - orig, -eps, eps)
      return proj

  ##
  # Main functions
  ##

  def make_adversarial_examples(self, image, true_label, args, model_to_fool, DATASET_SIZE):
      '''
      The main process for generating adversarial examples with priors.
      '''
      # Initial setup
      prior_size = DATASET_SIZE
      upsampler = Upsample(size=(DATASET_SIZE, DATASET_SIZE))
      total_queries = ch.zeros(args["batch_size"])
      prior = ch.zeros(args["batch_size"], self.channel_size, prior_size, prior_size)
      dim = prior.nelement()/args["batch_size"]
      image_step = self.l2_image_step if args["mode"] == 'l2' else self.linf_step
      proj_maker = self.l2_proj if args["mode"] == 'l2' else self.linf_proj
      proj_step = proj_maker(image, args["epsilon"])

      # Loss function
      criterion = ch.nn.CrossEntropyLoss(reduction='none')

      def normalized_eval(x):
          x_copy = x.clone()

          # You can comment or uncomment the code needed for a specific case
          """################## Normal prediction ##################"""
          # x_copy = ch.clamp(x_copy, 0, 1)
          # return self.model_to_fool(x_copy)

          """################## Prediction using random noise defense ##################"""
          x_copy_noisy = self.add_Gausian(x_copy, self.RND_coefficient)
          x_copy_noisy = ch.clamp(x_copy_noisy, 0, 1)
          return self.model_to_fool(x_copy_noisy)

      L = lambda x: criterion(normalized_eval(x), true_label)
      
      losses = L(image.to(device))

      # Original classifications
      orig_images = image.clone()
      
      # You can comment or uncomment the code needed for a specific case
      """##################Normal prediction##################"""
      # orig_classes = self.model_to_fool(orig_images).argmax(1).to(self.device)

      """################## Prediction using random noise defense ##################"""
      image_noisy = self.add_Gausian(image, self.RND_coefficient)
      image_noisy = ch.clamp(image_noisy, 0, 1)
      orig_classes = self.model_to_fool(image_noisy).argmax(1).to(self.device)

      correct_classified_mask = (orig_classes == true_label).float()
      total_ims = correct_classified_mask.sum()
      not_dones_mask = correct_classified_mask.clone()
      
      t = 0
      while not ch.any(total_queries > args["max_queries"]):
          t += args["gradient_iters"]*2
          if t >= args["max_queries"]:
              break

          prior = ch.zeros_like(image)
          for _ in range(args["gradient_iters"]):

              # Define a tensor of random Gaussian noise with the input image shape
              # Then devide this tensor by (dim**0.5)
              # Now add and subtract the defined noise with the fd_eta coefficient with the original image
              # Finally build the gradient estimation of the loss using the finite difference method
              #############################
              # Your code goes here
              fd_eta = args["fd_eta"]

              exp_noise = torch.randn_like(image) / (image.size(2) ** 0.5)


              noise_pos = exp_noise.clone()  
              noise_neg = exp_noise.clone()

              image_pos = image + fd_eta * noise_pos
              image_neg = image - fd_eta * noise_neg

              loss_pos = L(image_pos)
              loss_neg = L(image_neg)

              est_deriv = (loss_pos - loss_neg) / (2 * fd_eta)
              #############################

              prior += est_deriv.view(-1, 1, 1, 1)*exp_noise

          # Preserve images that are already done, 
          # Unless we are specifically measuring gradient estimation
          prior = prior*not_dones_mask.view(-1, 1, 1, 1)

          ## Update the image:
          # take a pgd step using the prior
          new_im = image_step(image, upsampler(prior*correct_classified_mask.view(-1, 1, 1, 1)), args["image_lr"])
          image = proj_step(new_im)
          image = ch.clamp(image, 0, 1)

          if args["mode"] == 'l2':
              if not ch.all(self.norm(image - orig_images) <= args["epsilon"] + 1e-3):
                  pass
                  # pdb.set_trace()
          else:
              if not (image - orig_images).max() <= args["epsilon"] + 1e-3:
                pass
                  # pdb.set_trace()

          ## Continue query count
          total_queries += 2*args["gradient_iters"]*not_dones_mask
          not_dones_mask = not_dones_mask*((normalized_eval(image).argmax(1) == true_label).float())

          ## Logging and stuff

          # new_losses = L(image)
          success_mask = (1 - not_dones_mask)*correct_classified_mask
          num_success = success_mask.sum()

          if num_success != 0:
            success_queries = ((success_mask*total_queries).sum()/num_success).cpu().item()
          else:
            success_queries = ((success_mask*total_queries).sum()).cpu().item()
          """If num_success == 0 it means that the number of successful adv examples was zero (No adv examples could be made!)
          so success queries must not be printed """

          current_success_rate = (num_success/correct_classified_mask.sum()).cpu().item()
          #not_done_loss = ((new_losses*not_dones_mask).sum()/not_dones_mask.sum()).cpu().item()
          max_curr_queries = total_queries.max().cpu().item()
          if args["log_progress"]:
            print("Queries: %d (%d) | Success rate: %f | Average queries: %f" % (max_curr_queries , t, current_success_rate, success_queries))

          if current_success_rate == 1.0:
              break

      return {
              'average_queries': success_queries,
              'num_correctly_classified': correct_classified_mask.sum().cpu().item(),
              'success_rate': current_success_rate,
              'images_orig': orig_images.cpu().numpy(),
              'images_adv': image.cpu().numpy(),
              'all_queries': total_queries.cpu().numpy(),
              'correctly_classified': correct_classified_mask.cpu().numpy(),
              'success': success_mask.cpu().numpy()
      }

# **Attack Arguments Setup**

**Write a report and indicate the success rate and average number of qureies for sections A and B.**

A) Now you must launch the attack using different values;
For the whole CIFAR10 test set, first on the regular target model and without the Random Noise Defense, with fd_eta parameter of 0.01 and 0.001, using "linf" mode (2 results). 

B) Then apply the Random Noise Defense on the target model with RND_coefficient of 0.01 and 0.02 against NES with fd_eta parameter of 0.01 and 0.001 again using the "linf" mode (4 results).

Analyse and compare the results of both sections in your report.


In [ ]:
budget = 1000
target_model_arch = 'resnet18' 
dataset_size = 32

args = {'max_queries': budget,
              'fd_eta': 0.01,
              'image_lr': 0.01,
              'mode': 'linf',
              'json_config': None,
              'epsilon': 0.05,
              'batch_size': 100,
              'log_progress': "store_False",
              'gradient_iters': 15,
              'total_images': 10000,
              'classifier': target_model_arch,
              }


RND_coefficient = 0.01  # Random Noise Defense coefficient
device = 'cuda'  # or 'cpu' depending on your available hardware
target_model = net.cuda()
target_model.eval()

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here

    fd_etas = [0.01]

    # Section A
    success_rates_a = []
    avg_queries_a = []
    for fd_eta in fd_etas:
        args['fd_eta'] = fd_eta
        attack = NES(target_model, args, dataset_size, 0, device)
        attack.generate()



    #############################

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Queries: 30 (30) | Success rate: 0.072165 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.113402 | Average queries: 40.909092
Queries: 90 (90) | Success rate: 0.164948 | Average queries: 56.250000
Queries: 120 (120) | Success rate: 0.247423 | Average queries: 77.500000
Queries: 150 (150) | Success rate: 0.340206 | Average queries: 97.272728
Queries: 180 (180) | Success rate: 0.412371 | Average queries: 111.750000
Queries: 210 (210) | Success rate: 0.484536 | Average queries: 126.382980
Queries: 240 (240) | Success rate: 0.556701 | Average queries: 141.111115
Queries: 270 (270) | Success rate: 0.597938 | Average queries: 150.000000
Queries: 300 (300) | Success rate: 0.649485 | Average queries: 161.904755
Queries: 330 (330) | Success rate: 0.670103 | Average queries: 167.076920
Queries: 360 (360) | Success rate: 0.711340 | Average queries: 178.260864
Queries: 390 (390) | Success rate: 0.711340 | Average queries: 178.260864
Queries: 420 (420) | Success rate: 0.731959 | Ave

TypeError: ignored

In [ ]:
budget = 1000
target_model_arch = 'resnet18' 
dataset_size = 32

args = {'max_queries': budget,
              'fd_eta': 0.01,
              'image_lr': 0.01,
              'mode': 'linf',
              'json_config': None,
              'epsilon': 0.05,
              'batch_size': 100,
              'log_progress': "store_False",
              'gradient_iters': 15,
              'total_images': 10000,
              'classifier': target_model_arch,
              }


RND_coefficient = 0.01  # Random Noise Defense coefficient
device = 'cuda'  # or 'cpu' depending on your available hardware
target_model = net.cuda()
target_model.eval()

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here

    fd_etas = [ 0.001]

    for fd_eta in fd_etas:
        args['fd_eta'] = fd_eta
        attack = NES(target_model, args, dataset_size, 0, device)
        attack.generate()

    #############################

Queries: 30 (30) | Success rate: 0.072165 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.113402 | Average queries: 40.909092
Queries: 90 (90) | Success rate: 0.144330 | Average queries: 51.428570
Queries: 120 (120) | Success rate: 0.226804 | Average queries: 76.363640
Queries: 150 (150) | Success rate: 0.319588 | Average queries: 97.741936
Queries: 180 (180) | Success rate: 0.412371 | Average queries: 116.250000
Queries: 210 (210) | Success rate: 0.474227 | Average queries: 128.478256
Queries: 240 (240) | Success rate: 0.505155 | Average queries: 135.306122
Queries: 270 (270) | Success rate: 0.577320 | Average queries: 152.142853
Queries: 300 (300) | Success rate: 0.608247 | Average queries: 159.661011
Queries: 330 (330) | Success rate: 0.628866 | Average queries: 165.245895
Queries: 360 (360) | Success rate: 0.628866 | Average queries: 165.245895
Queries: 390 (390) | Success rate: 0.670103 | Average queries: 179.076920
Queries: 420 (420) | Success rate: 0.680412 | Ave

In [10]:
budget = 1000
target_model_arch = 'resnet18' 
dataset_size = 32

args = {'max_queries': budget,
              'fd_eta': 0.01,
              'image_lr': 0.01,
              'mode': 'linf',
              'json_config': None,
              'epsilon': 0.05,
              'batch_size': 100,
              'log_progress': "store_False",
              'gradient_iters': 15,
              'total_images': 10000,
              'classifier': target_model_arch,
              }
target_model = net.cuda()
target_model.eval()


with torch.no_grad():
    RND_coefficients = [0.01, 0.02]
    fd_etas = [0.01, 0.001]

    for RND_coefficient in RND_coefficients:
        for fd_eta in fd_etas:
            args['fd_eta'] = fd_eta
            nes = NES(target_model, args, dataset_size, RND_coefficient, device)
            nes.generate()


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Queries: 150 (150) | Success rate: 0.136842 | Average queries: 76.153847
Queries: 180 (180) | Success rate: 0.136842 | Average queries: 76.153847
Queries: 210 (210) | Success rate: 0.157895 | Average queries: 94.000000
Queries: 240 (240) | Success rate: 0.157895 | Average queries: 94.000000
Queries: 270 (270) | Success rate: 0.168421 | Average queries: 105.000000
Queries: 300 (300) | Success rate: 0.189474 | Average queries: 126.666664
Queries: 330 (330) | Success rate: 0.189474 | Average queries: 126.666664
Queries: 360 (360) | Success rate: 0.200000 | Average queries: 138.947372
Queries: 390 (390) | Success rate: 0.210526 | Average queries: 151.500000
Queries: 420 (420) | Success rate: 0.221053 | Average queries: 164.285721
Queries: 450 (450) | Success rate: 0.231579 | Average queries: 177.272720
Queries: 480 (480) | Success rate: 0.242105 | Average queries: 190.434784
Queries: 510 (510) | Success rate: 0.252632 | Average queries: 20

Overall results:

**without defence with fd_eta =  0.01**

Final Success Rate: 0.915578309875112 | Final Average Queries for Successful Adv Examples: 270
****
        
**without defence with fd_eta =  0.001**

Final Success Rate: 0.9124438399407149 | Final Average Queries for Successful Adv Examples: 274
 ****

**with defence with fd_eta =  0.01   ,RND_coefficient =  0.01**

Final Success Rate: 0.4237288148425255 | Final Average Queries for Successful Adv Examples: 359
 ****

**with defence with fd_eta =  0.001   ,RND_coefficient =  0.01**

Final Success Rate: 0.2327586194505027 | Final Average Queries for Successful Adv Examples: 330
 ****

**with defence with fd_eta =  0.01   ,RND_coefficient =  0.02**

Final Success Rate: 0.3635773086390386 | Final Average Queries for Successful Adv Examples: 332
 ****
        

**with defence with fd_eta =  0.001   ,RND_coefficient =  0.02**

Final Success Rate: 0.294472578927022 | Final Average Queries for Successful Adv Examples: 316

 ****
       
